In [1]:
import spacy
import pandas as pd
import numpy as np

import Modules.solrhandler as sh
import Modules.clusterer as cls
import Modules.topicdeterminator as td
import Modules.chatbot as cb

In [2]:
#query = "anmeldung"
#target_service = 120686

import warnings
warnings.filterwarnings(action='ignore')

In [3]:
# Import eval.csv
import os
file = os.path.realpath("../../Evaluierung/eval.csv")
eval = pd.read_csv(file, sep = ";")
eval = eval.head(1000)
eval.head()

,documentId,file,dialogId,initialQuestion,suggestionCount,directlyFound,name
0,120697,20220131--quantEvalAllExcel.xlsx,4,Ummeldung,6,True,Änderung/Wechsel der Hauptwohnung
1,120686,20220131--quantEvalAllExcel.xlsx,4,Ummelden,6,True,Anmeldung einer Wohnung
2,329421,20220131--quantEvalAllExcel.xlsx,19,Pcr test positiv brauche ich eine Krankschreib...,6,False,Erstattung nach Infektionsschutzgesetz bei Tät...
3,329421,20220131--quantEvalAllExcel.xlsx,19,Quarantäne Krankschreibung vom Arzt,6,True,Erstattung nach Infektionsschutzgesetz bei Tät...
4,120914,20220131--quantEvalAllExcel.xlsx,23,Termin für Zulassung Gebrauchtwagen,6,True,Zulassung eines Fahrzeuges mit auswärtigem Ken...


Funktionen

In [4]:
def findCorrectAnswer(chatbot,targetService):

    # Find Target Cluster
    clusteredColumn = chatbot.clusterer.getClusteredColumn()
    service = chatbot.df.loc[chatbot.df["id"]==str(targetService)]
    if len(service[clusteredColumn].values) == 0:
        return None

    targetCluster = service[clusteredColumn].values[0]

    if targetCluster == chatbot.getSelectedClusterForQuestion():
        return True

    else:
        return False

def getRank(chatbot, targetService):
    temp = chatbot.df#.reset_index()
    rank = temp.index[temp["id"]==str(targetService)].values[0]
    return rank

Logging

In [5]:
class Logger:
    
    def __init__(self, chatbot, eval_df):
        self.liste = []
        self.chatbot = chatbot
        self.eval_df = eval_df
    
    def getDataFrame(self): # TODO
        pass
    
    def log(self, ID, t, row, answer = None): 
        target_service = row["documentId"]
        
        dialogId = row["dialogId"]
        file = row["file"]
        
        service_name = row["name"]
        query = row["initialQuestion"]
        
        
        if answer is not None:

            
            nResults = len(self.chatbot.df.index)
            question = self.chatbot.generateQuestion()
            answer = answer

            rank = getRank(self.chatbot,target_service)# getRank(target_service)
            
            row = (file, dialogId, ID, t,  service_name, query, question, answer, rank, nResults)
        else:
            #nInitialResults = len(chatbot.df.index)
            #initialRank = np.nan
            row = (file, dialogId, ID, t, service_name, query, None, None, None, None)
        self.liste.append(row)
        #print(self.liste)

Execution

In [ ]:
# Initialisierung Komponenten Chatbot
solrhandler = sh.SolrHandler(10)
clusterer = cls.Clusterer()
tpc_dterminator = td.TopicDeterminator()
chatbot = cb.Chatbot(solrhandler,clusterer,tpc_dterminator)

In [6]:
# Experiment durchlaufen und Loggen
lg = Logger(chatbot,eval)   

for (ID, row) in eval.iterrows():
    
    # fun
    query = row["initialQuestion"]
    target_service = row["documentId"]
    
    # Log
    try:
        chatbot.initialQuery(query)
    except Exception as e:
        if(str(e) == 'response'): # TODO ??
            next
        elif(str(e) == 'no solr output'):
            next
        else:
            print(e)
    
    t = 0
    answer = findCorrectAnswer(chatbot, target_service)
    lg.log(ID,t, row, answer)
    if answer is None:
        continue
    
    
    while not chatbot.refineResultset(answer, recluster = True):
        # log
        t += 1
        
        answer = findCorrectAnswer(chatbot, target_service)
        if answer is None: 
            break
        lg.log(ID, t, row,answer)

In [7]:
#query
#len(solrhandler.get_df_from_query(query))

In [8]:
log_df = pd.DataFrame(lg.liste, columns=["file","dialogId", "ID","t", "service_name", "query", "question", "answer", "rank", "nResults"])
log_df["rank"] = log_df["rank"]+1
log_df["t"] = log_df["t"] +1
log_df.head()

,file,dialogId,ID,t,service_name,query,question,answer,rank,nResults
0,20220131--quantEvalAllExcel.xlsx,4,0,0,Änderung/Wechsel der Hauptwohnung,Ummeldung,Geht es bei ihrem Anliegen um zulassung?,True,3.0,7.0
1,20220131--quantEvalAllExcel.xlsx,4,1,0,Anmeldung einer Wohnung,Ummelden,None,None,NaN,NaN
2,20220131--quantEvalAllExcel.xlsx,19,2,0,Erstattung nach Infektionsschutzgesetz bei Tät...,Pcr test positiv brauche ich eine Krankschreib...,Geht es bei ihrem Anliegen um ablegen?,False,4.0,10.0
3,20220131--quantEvalAllExcel.xlsx,19,2,1,Erstattung nach Infektionsschutzgesetz bei Tät...,Pcr test positiv brauche ich eine Krankschreib...,Geht es bei ihrem Anliegen um sars?,True,4.0,6.0
4,20220131--quantEvalAllExcel.xlsx,19,2,2,Erstattung nach Infektionsschutzgesetz bei Tät...,Pcr test positiv brauche ich eine Krankschreib...,Geht es bei ihrem Anliegen um hinweiscrn?,False,2.0,3.0


Statistics

In [50]:
# avg Rank Improvement in turn t
log_df["rank_improvement"] = log_df.groupby(["ID"])["rank"].diff()
log_df.groupby(["t"])["rank_improvement"].mean()

t
1         NaN
2   -0.666667
3   -1.000000
4   -1.055556
5   -1.000000
6   -0.800000
7   -1.000000
Name: rank_improvement, dtype: float64

In [51]:
# avg nSuggestions Improvement in turn t
log_df["sugg_improvement"] = log_df.groupby(["ID"])["nResults"].diff()
log_df.groupby(["t"])["sugg_improvement"].mean()

t
1         NaN
2   -3.456140
3   -1.192982
4   -1.111111
5   -1.428571
6   -1.200000
7   -1.000000
Name: sugg_improvement, dtype: float64

In [48]:
# MRR ohne cluster refinement
df = log_df.loc[log_df["t"] == 1]
MRR_ohne = 1/(df["rank"].mean())
MRR_ohne

0.4828551434569629

In [49]:
# MRR mit cluster refinement
df = log_df.groupby(["ID"]).last("t")
MRR_mit = 1/(df["rank"].mean())
MRR_mit
#df.head()

0.5623471882640587

In [14]:
# avg Turns (Achtun!) -> chatbot geht noch nicht bis auf einzelnen service herunter
log_df["t"].mean()

1.3161239078633837

In [43]:
log_df.head(20)

,file,dialogId,ID,t,service_name,query,question,answer,rank,nResults,rank_improvement,sugg_improvement,MRR
0,20220131--quantEvalAllExcel.xlsx,4,0,1,Änderung/Wechsel der Hauptwohnung,Ummeldung,Geht es bei ihrem Anliegen um zulassung?,True,3.0,7.0,NaN,NaN,0.333333
1,20220131--quantEvalAllExcel.xlsx,4,1,1,Anmeldung einer Wohnung,Ummelden,None,None,NaN,NaN,NaN,NaN,NaN
2,20220131--quantEvalAllExcel.xlsx,19,2,1,Erstattung nach Infektionsschutzgesetz bei Tät...,Pcr test positiv brauche ich eine Krankschreib...,Geht es bei ihrem Anliegen um ablegen?,False,4.0,10.0,NaN,NaN,0.250000
3,20220131--quantEvalAllExcel.xlsx,19,2,2,Erstattung nach Infektionsschutzgesetz bei Tät...,Pcr test positiv brauche ich eine Krankschreib...,Geht es bei ihrem Anliegen um sars?,True,4.0,6.0,0.0,-4.0,0.250000
4,20220131--quantEvalAllExcel.xlsx,19,2,3,Erstattung nach Infektionsschutzgesetz bei Tät...,Pcr test positiv brauche ich eine Krankschreib...,Geht es bei ihrem Anliegen um hinweiscrn?,False,2.0,3.0,-2.0,-3.0,0.500000
5,20220131--quantEvalAllExcel.xlsx,19,3,1,Erstattung nach Infektionsschutzgesetz bei Tät...,Quarantäne Krankschreibung vom Arzt,Geht es bei ihrem Anliegen um approbation?,False,1.0,10.0,NaN,NaN,1.000000
6,20220131--quantEvalAllExcel.xlsx,19,3,2,Erstattung nach Infektionsschutzgesetz bei Tät...,Quarantäne Krankschreibung vom Arzt,Geht es bei ihrem Anliegen um cov?,True,1.0,3.0,0.0,-7.0,1.000000
7,20220131--quantEvalAllExcel.xlsx,23,4,1,Zulassung eines Fahrzeuges mit auswärtigem Ken...,Termin für Zulassung Gebrauchtwagen,Geht es bei ihrem Anliegen um kfz?,True,7.0,10.0,NaN,NaN,0.142857
8,20220131--quantEvalAllExcel.xlsx,24,5,1,Infektionskrankheiten melden - für Gemeinschaf...,Wo kann ich mich nach Corona Infektion freites...,None,None,NaN,NaN,NaN,NaN,NaN
9,20220131--quantEvalAllExcel.xlsx,28,6,1,Impfung gegen Corona (SARS-CoV-2),Ab wann ist man bei Johnson & Johnson geboostert,None,None,NaN,NaN,NaN,NaN,NaN
